walmart-sales/
├─ data/
│  ├─ raw/          # fichiers bruts
│  └─ processed/    # après prétraitement
├─ notebooks/
│  ├─ 01_eda.ipynb
│  ├─ 02_preprocessing_and_baseline.ipynb
│  └─ 03_regularized_models.ipynb
├─ src/
│  ├─ data_prep.py
│  ├─ models.py
│  └─ evaluate.py
├─ reports/
│  └─ walmart_report.md / .pdf
└─ README.md

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Part 1 — EDA + preprocessing (pandas)

In [47]:
df = pd.read_csv("https://julie-resources.s3.eu-west-3.amazonaws.com/full-stack-full-time/projects-supervised-machine-learning-ft/walmart-sales-ft/Walmart_Store_sales.csv")
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
mean,9.866667,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000
max,20.000000,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000


Bloc A — Supprimer les lignes où la cible manque

In [48]:
df = df.dropna(subset=["Weekly_Sales"])

In [49]:
df["Weekly_Sales"].isna().sum()

np.int64(0)

Bloc B — Transformer la colonne Date

In [50]:
df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["DayOfWeek"] = df["Date"].dt.weekday  # 0=lundi

/tmp/ipykernel_252468/1233037310.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Date"] = pd.to_datetime(df["Date"])


In [51]:
df.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day',
       'DayOfWeek'],
      dtype='object')

In [52]:
df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["DayOfWeek"] = df["Date"].dt.weekday  # 0=lundi


In [53]:
df = df.drop(columns=["Date"])

Gestion des valeurs aberrantes (Temp, Fuel_Price, CPI, Unemployment)

3 options : voir si je garde la précédente cellule ou celle-ci 

In [ ]:
# Before removing outliers
rows_before = df.shape[0]

num_cols_outliers = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

#for col in num_cols_outliers:
    mean = df[col].mean()
    std = df[col].std()
    lower = mean - 3 * std
    upper = mean + 3 * std
    df = df[(df[col] >= lower) & (df[col] <= upper)]

# After removing outliers
#rows_after = df.shape[0]

print("Rows before cleaning :", rows_before)
print("Rows after cleaning  :", rows_after)
print("Rows removed         :", rows_before - rows_after)


In [ ]:
#Version la plus simple (recommandée)
 
results = []

df_clean = df.copy()

for col in num_cols_outliers:
    before = df_clean.shape[0]
    mean = df_clean[col].mean()
    std = df_clean[col].std()
    lower = mean - 3 * std
    upper = mean + 3 * std
    
    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    after = df_clean.shape[0]
    
    results.append((col, before - after))

df = df_clean

for col, removed in results:
    print(f"Removed {removed} rows due to outliers in {col}")

Removed 0 rows due to outliers in Temperature
Removed 0 rows due to outliers in Fuel_Price
Removed 0 rows due to outliers in CPI
Removed 0 rows due to outliers in Unemployment


In [ ]:
#Version plus détaillée (par colonne) — si tu veux être “premium”

results = []

df_clean = df.copy()

for col in num_cols_outliers:
    before = df_clean.shape[0]
    mean = df_clean[col].mean()
    std = df_clean[col].std()
    lower = mean - 3 * std
    upper = mean + 3 * std
    
    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    after = df_clean.shape[0]
    
    results.append((col, before - after))

df = df_clean

for col, removed in results:
    print(f"Removed {removed} rows due to outliers in {col}")


Removed 15 rows due to outliers in Temperature
Removed 12 rows due to outliers in Fuel_Price
Removed 7 rows due to outliers in CPI
Removed 12 rows due to outliers in Unemployment


IndentationError: unexpected indent (3474465680.py, line 7)

Removed 0 rows due to outliers in Temperature
Removed 0 rows due to outliers in Fuel_Price
Removed 0 rows due to outliers in CPI
Removed 0 rows due to outliers in Unemployment
